In [1]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd

fl = list(glob.glob('../features/*.pkl'))
for l in sorted(fl):
    print("'{}'".format(l))

fl_tmp = list(glob.glob('../2nd_features/*.pkl'))
fl_2nd = []
for f in fl_tmp:
    if 'other' not in f:
        fl_2nd.append(f)
# no other feat7 

'../features/cnn_1_aug1_feat.pkl'
'../features/cnn_1_aug_1_fold8_feat.pkl'
'../features/cnn_1_aug_rescale_preprocess_feat.pkl'
'../features/cnn_1_aug_skimage_denoise_feat.pkl'
'../features/cnn_1_aug_skimage_preprocess_feat.pkl'
'../features/cnn_1_feat.pkl'
'../features/cnn_2_aug1_feat.pkl'
'../features/cnn_2_aug_1_fold8_feat.pkl'
'../features/cnn_2_aug_denoise_preprocess_feat.pkl'
'../features/cnn_2_aug_rescale_preprocess_feat.pkl'
'../features/cnn_2_aug_skimage_preprocess_feat.pkl'
'../features/cnn_2_feat.pkl'
'../features/cnn_3_aug1_feat.pkl'
'../features/cnn_3_aug_1_fold8_feat.pkl'
'../features/cnn_3_aug_denoise_preprocess_feat.pkl'
'../features/cnn_3_aug_rescale_preprocess_feat.pkl'
'../features/cnn_3_aug_skimage_preprocess_feat.pkl'
'../features/cnn_4_aug1_feat.pkl'
'../features/cnn_4_aug1_feat_add_early.pkl'
'../features/cnn_4_aug_1_fold8_feat.pkl'
'../features/cnn_4_aug_denoise_preprocess_feat.pkl'
'../features/cnn_4_aug_rescale_preprocess_feat.pkl'
'../features/cnn_4_aug_skimag

In [2]:
print(fl_2nd)

['../2nd_features/cnn_4_aug_1_fold5_feat.pkl', '../2nd_features/cnn_2_aug_1_fold5_feat.pkl', '../2nd_features/cnn_5_aug_1_fold5_feat.pkl', '../2nd_features/cnn_3_aug_1_fold5_feat.pkl', '../2nd_features/vgg_aug1_new_feat.pkl', '../2nd_features/cnn_1_aug_1_fold5_feat.pkl', '../2nd_features/incept_aug2_feat.pkl', '../2nd_features/resnet_aug2_feat.pkl']


In [3]:
def load_x_y():
    train_x,test_x=[],[]
    for f in fl+fl_2nd:
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

(1604, 89) (8424, 89) (1604,)


In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False,verbose_cnt=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        params = {
                'colsample_bytree': 0.75,
                'colsample_bylevel':0.95,
                'gamma':0.2,
                'subsample': 0.9,
                'eta': 0.07,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight': 0.85,
                
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(test_x)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 500, watchlist, 
                        early_stopping_rounds=20,
                        verbose_eval=verbose_cnt
                        )
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict(d_valid))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            w_test_pred = m.predict(d_test)*curr_r_val_loss
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred
        print(submission.head())
        submission.to_csv('../results/all_xgb2nd_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        


    
print('def done')

def done


In [5]:
# find new rnd seed
# for i in range(50):
#     rnd = i*6
#     print(rnd)
#     cv_test(3,rnd,False)

In [7]:
# find new rnd seed
#for i in range(50):
#     rnd = i*6
#     print(rnd)
#     cv_test(5,rnd,False)

In [10]:
cv_test(3,42,True,500)

[0]	train-logloss:0.635959	valid-logloss:0.639502
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[163]	train-logloss:0.022057	valid-logloss:0.12258

[0]	train-logloss:0.635409	valid-logloss:0.637822
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[113]	train-logloss:0.036094	valid-logloss:0.137409

[0]	train-logloss:0.635145	valid-logloss:0.638074
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[80]	train-logloss:0.054254	valid-logloss:0.144867

local average valid loss 0.136257547104 val loss std 0.00872506165801
         id  is_iceberg
0  5941774d    0.018934
1  4023181e    0.908467
2  b20200e4    0.095

In [8]:
cv_test(3,252,True,500)

[0]	train-logloss:0.634653	valid-logloss:0.638591
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[95]	train-logloss:0.047384	valid-logloss:0.132299

[0]	train-logloss:0.635776	valid-logloss:0.637622
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[73]	train-logloss:0.057514	valid-logloss:0.150218

[0]	train-logloss:0.63577	valid-logloss:0.638985
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[84]	train-logloss:0.053877	valid-logloss:0.135554

local average valid loss 0.140004914885 val loss std 0.0081001101715
         id  is_iceberg
0  5941774d    0.029824
1  4023181e    0.872617
2  b20200e4    0.058897

In [9]:
cv_test(5,66,True,500)

[0]	train-logloss:0.636274	valid-logloss:0.640419
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[120]	train-logloss:0.041879	valid-logloss:0.121948

[0]	train-logloss:0.635221	valid-logloss:0.63957
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[114]	train-logloss:0.039663	valid-logloss:0.157246

[0]	train-logloss:0.635814	valid-logloss:0.636901
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[63]	train-logloss:0.073822	valid-logloss:0.137635

[0]	train-logloss:0.635556	valid-logloss:0.638941
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid